In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.cm as cm

import holodeck as holo
from holodeck import plot, utils
from holodeck.constants import YR

# Build Semi-Analytic Model, sam
Select a shape (smaller for speed, larger for accuracy, best with None) and use defaults for all other parameters (GSMF, GMT, MMbulge) in sam

In [ ]:
# SHAPE = None # default shape, if None, is 91,81,101.
SHAPE = 40  # make this smaller to run faster, for debugging

# get PTA frequencies
fobs_gw_cents, fobs_gw_edges = utils.pta_freqs()

# build Semi-Analytic Model
sam = holo.sams.Semi_Analytic_Model(shape=SHAPE)  

# Calculate SS and GWB strain with sam.gwb()
This function calls single_sources.ss_gws_redz().

### Choose a hardening model

In [ ]:
# uncomment one of the following hardening models

# # GW only hardening
hard=holo.hardening.Hard_GW()

# # Fixed Time (phenomenological) hardening
# hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)

assert isinstance(hard, (holo.hardening.Fixed_Time_2PL_SAM, holo.hardening.Hard_GW))

### Set the number of realizations and loudest sources

In [ ]:
NREALS = 30 # Set the number of strain realizations
NLOUDEST = 10 # set the number of loudest single sources to distinguish at each frequency

### Calculate the characteristic strain of the single sources and background
* hc_ss is shape [F,R,L] = [nfrequencies, nreals, nloudest], and gives the characteristic strain of each individual loudest source
* hc_bg is shape [F,R] and gives the sum of the char strain from all other sources at each frequency

In [ ]:

hc_ss, hc_bg, = sam.gwb(fobs_gw_edges, hard=hard, 
                        realize = NREALS, loudest = NLOUDEST)

### Plot

In [ ]:
nsamp = 5

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel = plot.LABEL_CHARACTERISTIC_STRAIN)

xx = fobs_gw_cents*YR
yy = 1e-15 * np.power(xx, -2.0/3.0)

# plot power law, for comparison
ax.plot(xx, yy, 'k--', alpha=0.5, lw=2.0)

# plot background median and confidence intervals
ax.plot(xx, np.median(hc_bg, axis=-1), color='tab:blue', alpha = 0.9)
for pp in [50, 95]:
    percs = [50-pp/2, 50+pp/2]
    ax.fill_between(xx, *np.percentile(hc_bg, percs, axis=-1), alpha=0.25, color='tab:blue')

# plot random realizations
nsamp = np.min([nsamp, NREALS])
colors = cm.rainbow(np.linspace(0,1,nsamp))
idx = np.random.choice(NREALS, nsamp, replace=False)

ci = 0
for ii in idx:
    # background
    ax.plot(xx, hc_bg[:,ii], ':', color=colors[ci], alpha=0.75)

    # single sources
    ax.scatter(np.repeat(xx, NLOUDEST), hc_ss[:,ii,:].flatten(), alpha=0.25, color=colors[ci])
    ci += 1

# Add nHz axis
plot._twin_hz(ax, nano=True)